<a href="https://colab.research.google.com/github/ilank-pro/SmartHotel360-Website/blob/master/Stock_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

# Step 1: Retrieve the list of Nasdaq-100 stocks
nasdaq_100_tickers = """
AAPL MSFT AMZN NVDA GOOG GOOGL META TSLA PEP ADBE NFLX CSCO AVGO TXN CMCSA COST INTU SBUX AMGN QCOM AMD BKNG GILD FISV ADP MU MDLZ LRCX ADI HON LLY AMAT ISRG PYPL NOW REGN PANW ORLY TMUS CHTR ABNB ILMN ZM SNPS MRVL FTNT KLAC DXCM NXPI VRTX MS DELL BIIB WDAY EA EXC SEDG MCHP PDD ASML IDXX KDP MAR WMT CDNS ALGN MRNA OKTA PTON DOCU TEAM ZS SPLK MDB DDOG SHOP CRSP NET U DKNG SQ PATH BILL ROKU TWLO RIVN CRWD
"""

# Convert the string of tickers into a list
tickers = nasdaq_100_tickers.split()

# Step 2: Fetch historical stock price data with error handling
valid_tickers = []
data_dict = {}
for ticker in tickers:
    try:
        data = yf.download(ticker, start="2024-01-01", end="2024-05-17")
        if not data.empty:
            valid_tickers.append(ticker)
            data_dict[ticker] = data
    except Exception as e:
        print(f"Failed to download data for {ticker}: {e}")



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [5]:
# Function to detect breakout based on multiple indicators
def detect_breakouts(data, volume_threshold=1.5, ma_window=50, rsi_window=14, rsi_threshold=70):
    breakout_stocks = []
    for ticker, stock_data in data.items():
        stock_data['Volume_MA'] = stock_data['Volume'].rolling(window=ma_window).mean()
        stock_data['Close_MA'] = stock_data['Close'].rolling(window=ma_window).mean()
        stock_data['RSI'] = compute_rsi(stock_data['Close'], window=rsi_window)

        recent_data = stock_data.iloc[-3:]  # Last 3 days
        if all(recent_data['Close'] > recent_data['Close_MA']) and \
           all(recent_data['RSI'] > rsi_threshold) and \
           any(recent_data['Volume'] > volume_threshold * recent_data['Volume_MA']):
            breakout_stocks.append(ticker)
    return breakout_stocks

# Function to compute RSI
def compute_rsi(series, window):
    delta = series.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Step 3: Calculate breakouts using multiple indicators
breakout_stocks = detect_breakouts(data_dict)
print (breakout_stocks)


[]


In [3]:
# Step 4: Filter data for the last 3 months and plot candlestick charts for the selected stocks
three_months_ago = datetime.now() - timedelta(days=90)
for ticker in breakout_stocks:
    stock_data = data_dict[ticker][['Open', 'High', 'Low', 'Close']].dropna()
    stock_data = stock_data[stock_data.index >= three_months_ago]

    fig = make_subplots(rows=1, cols=1)

    fig.add_trace(go.Candlestick(
        x=stock_data.index,
        open=stock_data['Open'],
        high=stock_data['High'],
        low=stock_data['Low'],
        close=stock_data['Close'],
        name=ticker
    ))

    fig.update_layout(
        title=f'Candlestick Chart for {ticker}',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False,
        width=1000,  # Increase the width for bigger candles
        height=600,  # Increase the height for bigger candles
        margin=dict(l=20, r=20, t=40, b=20),
        font=dict(size=14)
    )

    fig.show()
